# 6. Get question and answer pair

## 6.0 Load
return - **q** and **a**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import os
import sys
import time
start = time.time()

folderpath = '/content/gdrive/My Drive/iss/Capstone/PLP Project/iTechQnA'
os.chdir(folderpath)

In [3]:
!pip install pyspark

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.getOrCreate()

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 14.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=051db1c8d7abd15ffe9b62d1d3712aa9b542f4d1f4a02778f7728526ef875158
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [4]:
%%time
q = spark.read.format("json").load('/content/gdrive/My Drive/iss/Capstone/PLP Project/Dataset/data/q*txt')
q.printSchema()

root
 |-- backoff: long (nullable = true)
 |-- has_more: boolean (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- accepted_answer_id: long (nullable = true)
 |    |    |-- answer_count: long (nullable = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- bounty_amount: long (nullable = true)
 |    |    |-- bounty_closes_date: long (nullable = true)
 |    |    |-- closed_date: long (nullable = true)
 |    |    |-- closed_reason: string (nullable = true)
 |    |    |-- community_owned_date: long (nullable = true)
 |    |    |-- content_license: string (nullable = true)
 |    |    |-- creation_date: long (nullable = true)
 |    |    |-- is_answered: boolean (nullable = true)
 |    |    |-- last_activity_date: long (nullable = true)
 |    |    |-- last_edit_date: long (nullable = true)
 |    |    |-- link: string (nullable = true)
 |    |    |-- locked_date: long (nullable = true)
 |    |    |-- migrated_fro

In [5]:
%%time
a = spark.read.format("json").load('/content/gdrive/My Drive/iss/Capstone/PLP Project/Dataset/data/a*txt')
a.printSchema()

root
 |-- backoff: long (nullable = true)
 |-- has_more: boolean (nullable = true)
 |-- items: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- answer_id: long (nullable = true)
 |    |    |-- body: string (nullable = true)
 |    |    |-- community_owned_date: long (nullable = true)
 |    |    |-- content_license: string (nullable = true)
 |    |    |-- creation_date: long (nullable = true)
 |    |    |-- is_accepted: boolean (nullable = true)
 |    |    |-- last_activity_date: long (nullable = true)
 |    |    |-- last_edit_date: long (nullable = true)
 |    |    |-- owner: struct (nullable = true)
 |    |    |    |-- accept_rate: long (nullable = true)
 |    |    |    |-- display_name: string (nullable = true)
 |    |    |    |-- link: string (nullable = true)
 |    |    |    |-- profile_image: string (nullable = true)
 |    |    |    |-- reputation: long (nullable = true)
 |    |    |    |-- user_id: long (nullable = true)
 |    |    |    |-- us

## 6.1 Set parameters


In [6]:
# set score threshold
thres_score = 2
# set least count threshold
min_tags_freq = 100
min_tags_count = 3

# q is_answered = True
# a is_accepted = True

## 6.2 Filter columns and rows
return - **q_sub** and **a_sub**

In [7]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf

def select_q(q, 
             thres_score = thres_score,
             min_tags_freq = min_tags_freq,
             min_tags_count = min_tags_count):
  
  tmp = q.withColumn('items', F.explode('items'))  
  subtmp = tmp.select(F.col('items.question_id').alias('q_qid'),                     
                      F.to_date(F.from_unixtime('items.creation_date',"yyyy-MM-dd")).alias('q_creation_date'),                     
                      F.from_unixtime('items.creation_date',"yyyy").alias('q_year'),
                      F.col('items.score').alias('q_score'),
                      F.col('items.title').alias('q_title'),
                      F.col('items.body').alias('q_body'),                      
                      F.col('items.tags').alias('q_tags'),
                      F.col('items.is_answered').alias('q_is_answered')
                      )   
  print(f'Original no. of rows == {subtmp.count()}\n')
  # filter by is_answered == True
  subtmp = subtmp.select(['*']).where(F.col('q_is_answered') == True) 
  # filter by score
  subtmp = subtmp.select(['*']).where(F.col('q_score') >= thres_score)
  subtmp = subtmp.withColumn('no_of_tags', F.size('q_tags'))                   
  print(f'No. of rows with score >= {thres_score} is {subtmp.count()}\n')
  # tags group freq count
  freq = subtmp.groupBy('q_tags').count() \
          .orderBy(F.col('count').desc())
  freq.show()
  # left join
  subtmp  = subtmp.join(freq, subtmp.q_tags == freq.q_tags ,'left') \
          .select(subtmp['*'], freq['count'])
  subtmp.printSchema()
  # filter by tags freq and count
  subtmp = subtmp.select(['*']) \
                  .where((F.col('count') >= min_tags_freq) & (F.col('no_of_tags') >= min_tags_count))                    
  print(f'No. of rows after filtering is {subtmp.count()}\n')
  subtmp.printSchema()
  subtmp.show()

  return subtmp

In [8]:
q_sub = select_q(q)

Original no. of rows == 878471

No. of rows with score >= 2 is 118164

+--------------------+-----+
|              q_tags|count|
+--------------------+-----+
|    [python, pandas]| 5395|
|            [python]| 4283|
|[python, pandas, ...| 2827|
|[python, python-3.x]| 2200|
|     [python, numpy]| 1011|
|    [python, django]|  992|
|     [python, regex]|  789|
|[python, matplotlib]|  691|
|[python, tensorflow]|  656|
|[python, python-3...|  637|
|    [python, pygame]|  632|
|[python, pandas, ...|  531|
|      [python, list]|  479|
|[python, tensorfl...|  465|
|[python, arrays, ...|  448|
|[python, python-3...|  417|
|[python, pandas, ...|  415|
|[python, dictionary]|  314|
|[python, django, ...|  289|
|   [python, tkinter]|  264|
+--------------------+-----+
only showing top 20 rows

root
 |-- q_qid: long (nullable = true)
 |-- q_creation_date: date (nullable = true)
 |-- q_year: string (nullable = true)
 |-- q_score: long (nullable = true)
 |-- q_title: string (nullable = true)
 |-- q_b

In [9]:
def select_a(a):
  tmp = a.withColumn('items', F.explode('items'))  
  subtmp = tmp.select(F.col('items.question_id').alias('a_qid'),
                      F.col('items.answer_id').alias('a_aid'),                    
                      F.col('items.body').alias('a_body'),                      
                      F.col('items.is_accepted').alias('a_is_accepted')
                      )
  # filter by is_accepted == True
  subtmp = subtmp.select(['*']).where(F.col('a_is_accepted') == True)
  subtmp.printSchema()   
  return subtmp

In [10]:
a_sub = select_a(a)

root
 |-- a_qid: long (nullable = true)
 |-- a_aid: long (nullable = true)
 |-- a_body: string (nullable = true)
 |-- a_is_accepted: boolean (nullable = true)



## 6.3 Left join answers to questions
return **qa**

In [11]:
qa  = q_sub.join(a_sub, q_sub.q_qid == a_sub.a_qid ,'left')
qa.printSchema()

qa = qa.select(['*']).where(F.col('a_body').isNotNull())
qa = qa.drop('q_is_answered', 'no_of_tags', 'count', 'a_qid', 'a_is_accepted')
qa.printSchema()
qa.count()

root
 |-- q_qid: long (nullable = true)
 |-- q_creation_date: date (nullable = true)
 |-- q_year: string (nullable = true)
 |-- q_score: long (nullable = true)
 |-- q_title: string (nullable = true)
 |-- q_body: string (nullable = true)
 |-- q_tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- q_is_answered: boolean (nullable = true)
 |-- no_of_tags: integer (nullable = false)
 |-- count: long (nullable = true)
 |-- a_qid: long (nullable = true)
 |-- a_aid: long (nullable = true)
 |-- a_body: string (nullable = true)
 |-- a_is_accepted: boolean (nullable = true)

root
 |-- q_qid: long (nullable = true)
 |-- q_creation_date: date (nullable = true)
 |-- q_year: string (nullable = true)
 |-- q_score: long (nullable = true)
 |-- q_title: string (nullable = true)
 |-- q_body: string (nullable = true)
 |-- q_tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- a_aid: long (nullable = true)
 |-- a_body: string (nullable = true)



7613

In [12]:
import pandas as pd

df = qa.toPandas()
df.to_csv(folderpath + '/sample_data/qa.csv', index=False)

In [13]:
end = time.time()
print(end - start)

1500.4151904582977
